In [226]:
import os
import shutil
import xml.etree.ElementTree as ET

DATABASE_PATH = "./stanford_dogset/"
IMAGE_PATH = DATABASE_PATH+'Images/'
ANNOTATION_PATH = DATABASE_PATH+'Annotation/'
CLASS_PATH = './classes.txt'

In [227]:
def create_class():
    class_file = open('classes.txt','w')
    j = -1
    for name in os.listdir("./stanford_dogset/Annotation"):
        if name[0] == '.': continue
        print(name)
        j += 1
        for i in range(len(name)):
            if name[i] == '-':
                breed = name[i+1:]
                class_file.write(str(j)+': '+breed+'\n')
                break;
    class_file.close()
      
# create_class()

In [228]:
def make_database():
    for folder in os.listdir(IMAGE_PATH):
        if folder[0] == '.': continue
        for image in os.listdir(IMAGE_PATH+folder):
            shutil.move(IMAGE_PATH+folder+'/'+image, IMAGE_PATH+image)
    
# make_database()

In [229]:
def make_annotation():
    with open(CLASS_PATH) as f:
        class_names = f.readlines()
    classes = [c.strip() for c in class_names]
    classes = [c.split()[1] for c in classes]
    for folder in os.listdir(ANNOTATION_PATH):
        if folder[0] == '.': continue
        for annotation in os.listdir(ANNOTATION_PATH+folder):
            in_file = open(ANNOTATION_PATH+folder+'/'+annotation)
            tree = ET.parse(in_file)
            root = tree.getroot()
            list_file = open(ANNOTATION_PATH+annotation+'.txt','w')
            width = 0
            height = 0
            for sz in root.iter('size'):
                width = int(sz.find('width').text)
                height = int(sz.find('height').text)
            for obj in root.iter('object'):
                cls = obj.find('name').text
                cls_id = classes.index(cls)
                xmlbox = obj.find('bndbox')
                b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
                c = [(b[2]+b[0])/2.0/float(width), (b[1]+b[3])/2.0/float(height), (b[2]-b[0])/float(width), (b[3]-b[1])/float(height)]
                list_file.write(str(cls_id)+' '+' '.join([str(a) for a in c])+'\n');
            list_file.close()
            
# make_annotation()

In [230]:
def delete_file():
    for file in os.listdir(ANNOTATION_PATH):
        if file[-3:] == 'txt': os.remove(ANNOTATION_PATH+file)
        
# delete_file()

In [231]:
import scipy.io as sio
file_list = sio.loadmat('./stanford_dogset/lists/file_list.mat')
test_list = sio.loadmat('./stanford_dogset/lists/test_list.mat')
train_list = sio.loadmat('./stanford_dogset/lists/train_list.mat')

In [273]:
def split():
    IMAGE_PATH_NEW = DATABASE_PATH+'Images/'
    ANNOTATION_PATH_NEW = DATABASE_PATH+'Annotation/'
    
    if not os.path.exists(IMAGE_PATH_NEW+'test'): os.makedirs(IMAGE_PATH_NEW+'test')
    if not os.path.exists(ANNOTATION_PATH_NEW+'test'): os.makedirs(ANNOTATION_PATH_NEW+'test')
    for test in test_list['file_list']:
        path = IMAGE_PATH_NEW+test[0][0]
        filename = path.split('/')[-1]
        path = IMAGE_PATH_NEW+filename
        if os.path.exists(path): shutil.move(path,IMAGE_PATH_NEW+'test/'+filename)
        anno_path = ANNOTATION_PATH_NEW+filename[:-3]+'txt'
        if os.path.exists(anno_path): shutil.move(anno_path,ANNOTATION_PATH_NEW+'test/'+filename[:-3]+'txt')

    if not os.path.exists(IMAGE_PATH_NEW+'train'): os.makedirs(IMAGE_PATH_NEW+'train')
    if not os.path.exists(ANNOTATION_PATH_NEW+'train'): os.makedirs(ANNOTATION_PATH_NEW+'train')
    for train in train_list['file_list']:
        path = IMAGE_PATH_NEW+train[0][0]
        filename = path.split('/')[-1]
        path = IMAGE_PATH_NEW+filename
        if os.path.exists(path): shutil.move(path,IMAGE_PATH_NEW+'train/'+filename)
        anno_path = ANNOTATION_PATH_NEW+filename[:-3]+'txt'
        if os.path.exists(anno_path): shutil.move(anno_path,ANNOTATION_PATH_NEW+'train/'+filename[:-3]+'txt')
        
# split()